# Datenauswertung Treemotion Bosau, Messreihe 1

## Import Packages

In [ ]:
import pandas as pd

import treemotion
from treemotion import config, logger, db_manager, Projekt, Messreihe, Messung, Data, WindMessreihe

logger.setLevel(level="DEBUG")

## Daten-Import

Die Datenbank wurde bereits im Vorfeld angelegt.
Die Daten zum Messaufbau wurden bereits während der Feldarbeit manuell eingepflegt.
Teilweise wurden diese Daten manuell aus Google-Sheets übertragen, weil diese als Online-Anwendung im Baum effizient eingesetzt werden können.
Die "README_database_template_1_0_0_0_data_dictionary.md" enthält ein DataDictionary mit Beschreibung, Datentyp, Einheit etc.

In die folgenden Tabellen wurden die Daten manuell eingepflegt:

- Baum
- Baumart
- BaumBehandlung
- BaumBehandlungsVariante
- BaumKronensicherung
- BaumKronensicherungTyp
- Messreihe
- Messung (ohne Filenames)
- MessungStatus
- Projekt
- Sensor
- SensorOrt
- SensorStatus

In die folgenden Tabellen werden die Daten automatisiert in den folgenden Schritten durch das TreeMotion-Package importiert:

- Messung (filenames werden mit "treemotion.Projekt.add_filenames()" hinzugefügt).
- Data (enthält Metadaten zu jeder Messung; eine einzelne TMS-Messung kann mehrfach als Data-Objekt erstellt werden, um verschiedene verarbeitet Versionen einer Messung zu speichern; die Verarbeitung der Messung zu Versionen i. d. R. auf der Ebene vonn vollständigen Messreihen)
- auto_df_{version}_{id_messung}_messung (automatisch erstellt, enthält DataFrame eines jeden Data-Objekts)
- WindMessreihe (Winddaten werden mit "treemotion.MethodeWindMessreihe.load_from_dwd_online()" hinzugefügt)
- WindMessung (importierte Winddaten für relevanten Zeitraum bzw. Station "Dörnick"


### Datenbank-Verbindung, Projekt-Konfiguration und Messreihe

Lädt das Projekt inklusive der in Datenbank eingepflegten Metadaten.


In [ ]:
# Working Directory aus configuration.py übernommen

filename_datenbank = r"Bosau_2022.db"

db_manager.connect(db_filename=filename_datenbank)

# Erstellt Liste aller Projekte in Datenbank
projekt = Projekt.load_from_db(id_projekt=1)[0]

# Selektiere aus Gesamtprojekt die spezifische Messreihe
messreihe_1 = projekt.messreihen[0]


### Import Windmessdaten vom DWD

In [ ]:
windmessreihe = WindMessreihe.load_from_dwd_online(name="Doernick_2022", stations_id=6163, dec_auto_commit=True,
                                                   overwrite=False)

### Import Raw-CSV-Daten der Messreihe 1

Ergänzt auf Basis der Dateinamen und Metadaten in der Datenbank (id_sensor) die Dateipfade zu den Rohdaten.

In [ ]:
csv_path = r"C:\Users\mail\Meine Ablage\Kyellsen\005_Projekte\2022_Bosau\020_Daten"
projekt.add_filenames(csv_path)

messreihe_1.load_from_csv(version="raw", overwrite=False, dec_auto_commit=True)  # dec_auto_commit in Datenbank

## Daten-Exploration

Beispielhafter Einblick in einen Data-Frame innerhalb der ersten Messung als Rohdaten

In [ ]:
messung = messreihe_1.messungen[0]
data = messung.get_data_by_version(version="raw")

data.describe()

data.df.describe()

### Data-Validation

Hier nur einen erster Blick, welche Messung auffällige Fehler zeigen.
Entsprechend wird später selektiert, da ausreichend Messungen bzw. Zeiträume vorliegen

In [ ]:
# messreihe_1

## Data-Cleaning und Preparing

### Erstelle Arbeitsversion aus Rohdaten

Um Rohdaten nicht erneut importieren zu müssen, wird eine Kopie angelegt.

In [ ]:
messreihe_1_copy = messreihe_1.copy_data_by_version(version_source="raw", version_new="3600_sec_tempdrift_emd",
                                                    dec_auto_commit=False)

### Zeitraum sinnvoll eingrenzen

Die Methode findet in allen DataFrames eine definierte Anzahl Maxima in der "Absoluten Inclination" (n_peaks).
Zu beachten ist, dass hier auf Basis der default Drift-Kompensation von IML Electronics gearbeitet wird.
Dann wird für alle Messungen der gleiche Zeitraum ausgewählt (duration in Sekunden; hier 1h).
Der Zeitraum wird so gewählt, sodass die Anzahl der Peaks über alle Messung maximal ist.
Dadurch wird der Datenumfang reduziert und zugleich gewährleistet, dass ein für alle Messungen geeigneter Zeitraum mit hohen Neigungswerten vorliegt.


In [ ]:

messreihe_1_copy.limit_time_by_version_and_peaks(version="short_3600", duration=3600, show_peaks=True,
                                                 values_col='Absolute-Inclination - drift compensated', n_peaks=10,
                                                 min_time_diff=60, dec_auto_commit=True)



### Temperatur-Drift-Kompensation



## Sichere Trennung von Datenbank


In [ ]:
db_manager.disconnect()
